In [ ]:
import numpy as np
import pandas as pd
import vtk
import pyvista as pv
import scipy.io
import sys
import os

In [2]:
fileList = os.listdir()
mfiles = [i for i in fileList if i.endswith('.m') == True]
stlfiles = [i for i in fileList if i.endswith('.stl') == True]
stlfiles

['CoronaryArteryBranch.stl']

In [3]:
mesh = pv.read(stlfiles[0], file_format='stl')
points = mesh.points
points = np.array(points)

In [4]:
P1, P2, P3 = np.array([5, 0, 0]), np.array([0, 0, 5]), np.array([10, 0, 5])

In [5]:
def computeNormals(P1, P2, P3):
    top = np.cross((P2-P1), (P3-P1))
    return top/ np.sqrt(np.dot(top, top))


## Answer to Question 1
computeNormals(P1, P2, P3)

array([0., 1., 0.])

In [6]:
## Get Trianlges
triangles = mesh.triangulate()

In [7]:
## Get Normals

In [8]:
computeNormals(points[0], points[1], points[2])

array([ 0.54172257,  0.36500084, -0.75717306])